In [42]:
import torch

def NMS(boxes, scores, iou_threshold=0.5):
    
    scores = list(scores)
    order = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    
    x1, y1, x2, y2 = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]  # Coordinates
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    out = []

    while len(order) > 0:
        i = order[0]  
        out.append(order[0])
        
        if len(order) == 1:
            break
        
        xx1 = torch.max(x1[i], x1[torch.tensor(order[1:])])
        yy1 = torch.max(y1[i], y1[torch.tensor(order[1:])])
        xx2 = torch.min(x2[i], x2[torch.tensor(order[1:])])
        yy2 = torch.min(y2[i], y2[torch.tensor(order[1:])])
        
        inter = (xx2 - xx1 + 1).clamp(min=0) * (yy2 - yy1 + 1).clamp(min=0)
        iou = inter / (areas[i] + areas[torch.tensor(order[1:])] - inter)
        
        remaining_indices = [j for j, iou_value in enumerate(iou.tolist()) if iou_value <= iou_threshold]
        
        order = [order[j + 1] for j in remaining_indices]

    return out


In [43]:
boxes = torch.tensor([[100, 100, 210, 210], [105, 105, 215, 215], [150, 150, 250, 250]])
scores = torch.tensor([0.9, 0.75, 0.8])

keep_indices = NMS(boxes, scores, iou_threshold=0.5)
print( keep_indices)

[0, 2]


In [53]:
def compute_iou(box1, box2):
    x1, y1, x2, y2 = box1
    xx1, yy1, xx2, yy2 = box2

    # Compute intersection coordinates
    inter_x1 = max(x1, xx1)
    inter_y1 = max(y1, yy1)
    inter_x2 = min(x2, xx2)
    inter_y2 = min(y2, yy2)

    # Compute intersection area
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)

    # Compute areas of both boxes
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (xx2 - xx1) * (yy2 - yy1)

    # Compute union area
    union_area = box1_area + box2_area - inter_area

    # Compute IoU
    iou = inter_area / union_area
    return iou

In [66]:
from sklearn.metrics import precision_recall_curve, average_precision_score

def mAP(pred_boxes, gt_boxes, iou_threshold=0.5):
    y_true = []
    y_score = []
    matched_gt = {class_id: [False] * len(gt) for class_id, gt in enumerate(gt_boxes)}

    for pred in pred_boxes:
        xmin, ymin, xmax, ymax, score, class_id = pred
        matched = False
        best_match_idx = -1
        best_iou = 0.0

        if class_id < len(gt_boxes):
            for idx, gt in enumerate(gt_boxes[class_id]):
                if not matched_gt[class_id][idx]:  # Check if this GT box is not already matched
                    iou = compute_iou([xmin, ymin, xmax, ymax], gt)
                    if iou >= iou_threshold and iou > best_iou:
                        best_iou = iou
                        best_match_idx = idx
                        matched = True

            if matched:
                matched_gt[class_id][best_match_idx] = True
                y_true.append(1)  # True positive
            else:
                y_true.append(0)  # False positive
            y_score.append(score)

    # Handle unmatched ground truths (false negatives)
    for class_id, gt_list in enumerate(matched_gt):
        for matched in matched_gt[class_id]:
            if not matched:
                y_true.append(0)  # False negative
                y_score.append(0)  # Add a low score for completeness

    precision, recall, _ = precision_recall_curve(y_true, y_score)
    AP = average_precision_score(y_true, y_score)

    print("y_true:", y_true)
    print("y_score:", y_score)

    return AP



In [67]:
# Example usage
pred_boxes = [
    [50, 50, 150, 150, 0.9, 0],  # Predicted [xmin, ymin, xmax, ymax, score, class_id]
    [30, 30, 120, 120, 0.8, 1],
    # more predictions...
]
gt_boxes = [
    [[50, 50, 150, 150]],  # Ground truth for class 0
    [[50, 30, 120, 120]],  # Ground truth for class 1
    # more ground truths...
]

mAP = mAP(pred_boxes, gt_boxes)
print(mAP)

y_true: [1, 1]
y_score: [0.9, 0.8]
1.0
